# Data Driven Business project

- Elijah Hofman
- Jack Pieper
- Rick van der Kleij
- Mathijs de Jong (V2B)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from cleaning import *
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, mean_absolute_error
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor
from customAccuracy import *
import joblib

### Importeren dataset

In [ ]:
# Load the data
df = load_data('sap_storing_data_hu_project.csv')

# Data understanding

In [ ]:
df.describe()

# Data voorbereiding

In [ ]:
# Define the columns to drop
columns_to_drop = ['stm_sap_mon_meld_ddt', 'stm_mon_begin_ddt', 'stm_mon_toelichting_trdl', 'stm_oh_pg_mld',
                   'stm_scenario_mon', 'stm_mon_nr_status_omschr', 'stm_mon_nr__statuscode', 'stm_mon_nr_status_wijzdd',
                   'stm_aanntpl_ddt', 'stm_objectdl_code_gst', 'stm_objectdl_groep_gst', 'stm_progfh_in_ddt',
                   'stm_progfh_in_invoer_ddt', 'stm_progfh_gw_ddt', 'stm_progfh_gw_lwd_ddt', 'stm_progfh_hz',
                   'stm_veroorz_groep', 'stm_veroorz_code', 'stm_veroorz_tekst_kort', 'stm_effect', 'stm_afspr_aanvangddt',
                   'stm_mon_eind_ddt', 'stm_mon_vhdsincident', 'stm_dir_betrok_tr', 'stm_aangelegd_dd', 'stm_aangelegd_tijd',
                   'stm_mon_begindatum', 'stm_mon_begintijd', 'stm_progfh_gw_datum', 'stm_mon_eind_datum', 'stm_mon_eind_tijd',
                   'stm_controle_dd', 'stm_akkoord_mon_toewijz', 'stm_status_sapnaarmon', 'stm_fact_jn', 'stm_akkoord_melding_jn',
                   'stm_afsluit_ddt', 'stm_afsluit_dd', 'stm_afsluit_tijd', 'stm_rec_toegev_ddt', 'stm_hinderwaarde',
                   'stm_actie', 'stm_standplaats', 'stm_status_gebr', 'stm_wbi_nummer', 'stm_projnr', 'stm_historie_toelichting',
                   'stm_schade_verhaalb_jn', 'stm_schadenr', 'stm_schade_status_ga', 'stm_schade_statusdatum', 'stm_relatiervo_vorig',
                   'stm_relatiervo_volgend', 'stm_relatiervo', 'stm_afspr_func_hersteldd', 'stm_afspr_func_hersteltijd',
                   'stm_sorteerveld', 'stm_rapportage_maand', 'stm_rapportage_jaar', 'stm_x_bron_publ_dt', 'stm_x_bron_bestandsnaam',
                   'stm_x_bron_arch_dt', 'stm_x_actueel_ind', 'stm_x_run_id', 'stm_x_bk', 'stm_x_start_sessie_dt', 'stm_x_vervallen_ind']

# Drop the unnecessary columns
df = drop_columns(df, columns_to_drop)

# Clean the data (handle missing values, remove columns with excessive NaNs)
df, avg_list, mode_list = clean_data(df)

# Preprocess the data
df = preprocess_data(df)

# Save the cleaned data to a new CSV (optional)
save_data(df, 'final_db_cleaned.csv')

# Convert 'stm_aanntpl_tijd' to datetime if it is not already
df['stm_aanntpl_tijd'] = pd.to_datetime(df['stm_aanntpl_tijd'], format='%H:%M:%S')

# Calculate the number of seconds since the first date in the dataset
df['stm_aanntpl_tijd_in_seconden'] = (df['stm_aanntpl_tijd'] - df['stm_aanntpl_tijd'].min()).dt.total_seconds()

# Optionally display results
print("Data cleaning and preprocessing complete.")

In [ ]:
# Identificeer numerieke kolommen
numeric_columns = [col for col in df.columns if pd.api.types.is_numeric_dtype(df[col])]

# Toon het aantal ontbrekende waarden per kolom
print(df.isna().sum())

# Bereken de correlatiematrix voor numerieke kolommen en verwijder rijen/kolommen met alleen NaN-waarden
cmatrix = df[numeric_columns].corr().dropna(how='all', axis=1).dropna(how='all')

# Stel de grootte van de figuur in voor een grotere matrix
plt.figure(figsize=(14, 10))  # Pas deze waarden aan voor een nog groter formaat

# Plot de heatmap van de correlatiematrix
sns.heatmap(cmatrix, annot=True, fmt='.2f', cmap='coolwarm')
plt.show()
plt.close()


# target varible maken

In [ ]:
# Maak kolom voor de targetvariabele, de tijd tussen aannemer ter plaatse en functieherstel
df['stm_aanntpl_tijd'] = pd.to_datetime(df['stm_aanntpl_tijd'], format="%H:%M:%S", errors='coerce')

# Records waar stm_aanntpl_tijd niet is ingevuld verwijderen (stm_aanntpl_tijd == 00:00:00 in deze gevallen)
df = df[df['stm_aanntpl_tijd'] != pd.to_datetime("00:00:00").time()]

# Kolom targetvariabele aanmaken
df['target'] = df['stm_fh_ddt'] - df['stm_aanntpl_tijd']

# Selecteer alleen de tijd van target
df['target'] = df['target'].dt.components.apply(lambda x: f"{x.hours:02}:{x.minutes:02}:{x.seconds:02}", axis=1)

# Records met target van 0 minuten verwijderen (stm_aanntpl_tijd == stm_fh_ddt in deze gevallen)
df = df[df['target'] != pd.to_datetime("00:00:00").time()]

# Target converteren naar minuten zodat het in het model gebruikt kan worden
# Convert `target` to seconds
df['target'] = (df['target'] / pd.Timedelta(minutes=1)).astype(int)


df[['stm_sap_meld_ddt', 'stm_fh_ddt', 'totale_functiehersteltijd', 'stm_aanntpl_tijd', 'target', 'stm_fh_duur']].sample(10)

# Baseline model

In [ ]:
# Extract the relevant columns
X = df[['stm_progfh_in_duur_clean']]    # Predictor
y = df['target']                        # Target

# Handle missing values (if any)
X = X.fillna(X.mean())
y = y.fillna(y.mean())

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model's performance
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)


In [ ]:
# Onafhankelijke variabelen (X) en afhankelijke variabele (y)
X = df[['stm_progfh_in_duur_clean', 'stm_oorz_code']]
y = df['target']

# Splitsen van de dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Decision Tree Regressor aanmaken met beperkte diepte
dt_model = DecisionTreeRegressor(max_depth=3, random_state=42)

# Train het model
dt_model.fit(X_train, y_train)

# Maak voorspellingen op de testset
y_pred_dt = dt_model.predict(X_test)

# Bereken de evaluatiestatistieken
mse_dt = mean_squared_error(y_test, y_pred_dt)
mae_dt = mean_absolute_error(y_test, y_pred_dt)
rmse_dt = np.sqrt(mse_dt)

print(f'Mean Squared Error van Decision Tree: {mse_dt}') 
print(f'Mean Absolute Error van Decision Tree: {mae_dt}') 
print("Root Mean Squared Error (RMSE):", rmse_dt)

# Visualiseer de Decision Tree
plt.figure(figsize=(12, 8))
plot_tree(dt_model, feature_names=X.columns, filled=True)
plt.title('Decision Tree voor stm_fh_duur (max_depth=3)')
plt.show()


In [ ]:
# Onafhankelijke variabelen (X) en afhankelijke variabele (y)
X = df[['stm_progfh_in_duur_clean', 'stm_oorz_code', 'stm_geo_mld', 'stm_sap_meldtijd', 'stm_aanntpl_tijd_in_seconden']]
y = df['target']

# Splitsen van de dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Decision Tree Regressor aanmaken met beperkte diepte
dt2_model = DecisionTreeRegressor(max_depth=3, random_state=42)

# Train het model
dt2_model.fit(X_train, y_train)

# Maak voorspellingen op de testset
y_pred_dt = dt2_model.predict(X_test)

# Bereken de evaluatiestatistieken
mse2_dt = mean_squared_error(y_test, y_pred_dt)
mae2_dt = mean_absolute_error(y_test, y_pred_dt)
rmse2_dt = np.sqrt(mse2_dt)

print(f'Mean Squared Error van Decision Tree: {mse2_dt}') 
print(f'Mean Absolute Error van Decision Tree: {mae2_dt}') 
print("Root Mean Squared Error (RMSE):", rmse2_dt)

# Visualiseer de Decision Tree
plt.figure(figsize=(12, 8))
plot_tree(dt2_model, feature_names=X.columns, filled=True)
plt.title('Decision Tree voor stm_fh_duur (max_depth=3)')
plt.show()

In [ ]:
# Onafhankelijke variabelen (X) en afhankelijke variabele (y)
X = df[['stm_progfh_in_duur_clean', 'stm_oorz_code', 'stm_geo_mld', 'stm_sap_meldtijd', 'stm_aanntpl_tijd_in_seconden']]
y = df['target']

# Splitsen van de dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Instellen van de parameters voor Grid Search
param_grid = {
    'max_depth': [20],  # Verschillende diepten van de bomen
    'n_estimators': [100],  # We houden het aantal estimators constant op 100
}

# Grid Search aanmaken
grid_search = GridSearchCV(RandomForestRegressor(random_state=42), param_grid, cv=5, scoring='neg_mean_squared_error')

# Train het model met Grid Search
grid_search.fit(X_train, y_train)

# Beste model en parameters
best_rf_model = grid_search.best_estimator_
print(f'Beste hyperparameters: {grid_search.best_params_}')

# Sla het beste model op
joblib.dump(best_rf_model, 'beste_random_forest_model.pkl')

# Maak voorspellingen op de testset met het beste model
y_pred_rf = best_rf_model.predict(X_test)

# Bereken de evaluatiestatistieken
mse_rf = mean_squared_error(y_test, y_pred_rf)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)

print(f'Mean Squared Error van Decision Tree: {mse_rf}') 
print(f'Mean Absolute Error van Decision Tree: {mae_rf}') 
print("Root Mean Squared Error (RMSE):", rmse_rf)


# Belangrijkste kenmerken van het model plotten
importances = best_rf_model.feature_importances_
indices = np.argsort(importances)[::-1]
features = X.columns

plt.figure(figsize=(10, 6))
plt.title("Feature Importances in Random Forest Model")
plt.bar(range(X.shape[1]), importances[indices], align="center")
plt.xticks(range(X.shape[1]), [features[i] for i in indices], rotation=45)
plt.xlabel("Features")
plt.ylabel("Belang")
plt.show()

In [ ]:
# Limiteer de voorspellingen en werkelijke waarden
max_duration = 1500
y_pred_binned = np.clip(np.round(y_pred_rf / 10) * 10, 0, max_duration)
y_test_binned = np.clip(np.round(y_test / 10) * 10, 0, max_duration)

# Bereken de nauwkeurigheid binnen deze limiet
accuracy = accuracy_score(y_test_binned, y_pred_binned)
print(f"Nauwkeurigheid binnen 10-minuten-bins: {accuracy:.2f}")

# Unieke bins en frequenties berekenen in minuten
bins = np.unique(y_test_binned)
bin_counts = [(y_test_binned == bin).sum() for bin in bins if bin <= max_duration]

# Distributie van voorspellingen en werkelijke waarden plotten
plt.figure(figsize=(12, 6))
plt.hist(y_pred_binned, bins=np.arange(0, max_duration + 10, 10), alpha=0.5, label='Voorspellingen')
plt.hist(y_test_binned, bins=np.arange(0, max_duration + 10, 10), alpha=0.5, label='Werkelijke waarden')
plt.xlabel("Duur in minuten (gebundeld in 10-minuten-bins)")
plt.ylabel("Frequentie")
plt.legend()
plt.title("Distributie van voorspellingen en werkelijke waarden in 10-minuten-bins")
plt.show()